In [1]:
import scanpy as sc
sc.settings.n_jobs = -1
import h5py

In [ ]:
# I will fix this to dow load the h5 files later but i downlaoded them into a separate folder directly for now

# import GEOparse
# import os
# geo_id = "GSE234471"
# gse = GEOparse.get_GEO(geo=geo_id)
# # os.rename("../notebook_scripts/GSE234471_family.soft.gz", "../data/GSE234471_family.soft.gz")
# from urllib.request import urlretrieve
# url = "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE234471&format=file"
# path = "../data/GSE234471_processed.tar"
# urlretrieve(url, path)

# import tarfile 
# with open('file.', 'rb') as f_in:
#     with gzip.open('file.txt.gz', 'wb') as f_out:
#         f_out.writelines(f_in)

#downloaded h5 data directly from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE234471

('../data/GSE234471_processed.tar', <http.client.HTTPMessage at 0x12ce76b30>)

In [27]:
# Load the filtered feature/cell matrix from HDF5
c_data = sc.read_10x_h5("../data/GSM7469255_Veh-1_filtered_feature_bc_matrix.h5")
t_data = sc.read_10x_h5("../data/GSM7469256_dTAGV-1_filtered_feature_bc_matrix.h5")

/Users/Macbook/Desktop/scrna-portfolio/scrna_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/Macbook/Desktop/scrna-portfolio/scrna_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/Macbook/Desktop/scrna-portfolio/scrna_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/Macbook/Desktop/scrna-portfolio/scrna_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [20]:
with h5py.File('../data/GSM7469255_Veh-1_filtered_feature_bc_matrix.h5', 'r') as f:
    experiment_data = f['/matrix/barcodes'] 
    print(experiment_data.shape, experiment_data.dtype)

(13067,) |S18


In [21]:
print(adata.obs.head())

Empty DataFrame
Columns: []
Index: [AAACCCAAGACCAAGC-1, AAACCCAAGACTCCGC-1, AAACCCAAGCGTACAG-1, AAACCCAAGCTCGACC-1, AAACCCAAGTGAGGTC-1]


In [22]:
# convert top 50,000 datapoints of this to pandas df because, again, small virenv
import pandas as pd
import numpy as np

# tenary that converts matrix into dense if not already
dense_matrix = adata.X[:50000,].toarray() if hasattr(adata.X, 'toarray') else adata.X[:50000,]  

# Convert to Pandas DataFrame
df = pd.DataFrame(dense_matrix, index=adata.obs_names[:50000], columns=adata.var_names)

# Display the first few rows
print(df.head())

                    Xkr4  Gm1992  Gm19938  Gm37381  Rp1  Sox17  Gm37587  \
AAACCCAAGACCAAGC-1   0.0     0.0      0.0      0.0  0.0    0.0      0.0   
AAACCCAAGACTCCGC-1   0.0     0.0      0.0      0.0  0.0    0.0      0.0   
AAACCCAAGCGTACAG-1   0.0     0.0      0.0      0.0  0.0    0.0      0.0   
AAACCCAAGCTCGACC-1   0.0     0.0      0.0      0.0  0.0    0.0      0.0   
AAACCCAAGTGAGGTC-1   0.0     0.0      0.0      0.0  0.0    0.0      0.0   

                    Gm37323  Mrpl15  Lypla1  ...  Gm16367  AC163611.1  \
AAACCCAAGACCAAGC-1      0.0     0.0     0.0  ...      0.0         0.0   
AAACCCAAGACTCCGC-1      0.0     1.0     2.0  ...      0.0         0.0   
AAACCCAAGCGTACAG-1      0.0     0.0     0.0  ...      0.0         0.0   
AAACCCAAGCTCGACC-1      0.0     0.0     0.0  ...      0.0         0.0   
AAACCCAAGTGAGGTC-1      0.0     0.0     1.0  ...      0.0         0.0   

                    AC163611.2  AC140365.1  AC124606.2  AC124606.1  \
AAACCCAAGACCAAGC-1         0.0         0